# **Learning**

In [1]:
import pandas as pd
import numpy as np

from tensorflow import keras
import tensorflow as tf

/home/madbiker/miniconda3/envs/xai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/madbiker/miniconda3/envs/xai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/madbiker/miniconda3/envs/xai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/madbiker/miniconda3/envs/xai/lib/python3

## **Data Preparation**

In [2]:
DATA_SOURCE = "../01_data/adult_reconstruction.csv"

In [3]:
census_data = pd.read_csv(DATA_SOURCE)

In [4]:
census_data.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,gender,native-country,income,occupation
0,20,40,0,0,Private,Bachelors,13,Married-civ-spouse,Wife,White,Female,United-States,49100,Tech-support
1,40,21,0,0,Private,Some-college,10,Divorced,Own-child,White,Male,United-States,11500,Craft-repair
2,10,17,0,0,Private,11th,7,Never-married,Own-child,White,Male,United-States,2600,Other-service
3,50,51,0,0,Private,HS-grad,9,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,Cambodia,38997,Sales
4,38,28,0,0,Private,Bachelors,13,Never-married,Not-in-family,White,Male,?,41400,Exec-managerial


In [6]:
census_data["workclass"].unique()

array(['Private', '?', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov',
       'Local-gov', 'State-gov', 'Without-pay', 'Never-worked'],
      dtype=object)

In [18]:
x = census_data.drop(columns=["education", "income"]) #education: redundandt, income: label

#categorical encodings:
categorical = ["workclass", "marital-status", "relationship", "race", "gender", "native-country", "occupation"]
code_dictionary = {} #original values -> codes
for c in categorical:
    values = []
    #updating the code_dictionary:
    codes = census_data[c].unique()
    code_dictionary[c] = {}
    for i,code in enumerate(codes):
        code_dictionary[c][code] = i
    for _,r in x.iterrows():
        values.append(code_dictionary[c][r[c]])
    x[c] = values

In [19]:
x

,hours-per-week,age,capital-gain,capital-loss,workclass,education-num,marital-status,relationship,race,gender,native-country,occupation
0,20,40,0,0,0,13,0,0,0,0,0,0
1,40,21,0,0,0,10,1,1,0,1,0,1
2,10,17,0,0,0,7,2,1,0,1,0,2
3,50,51,0,0,0,9,0,2,1,1,1,3
4,38,28,0,0,0,13,2,3,0,1,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...
49526,65,35,0,0,0,13,0,2,0,1,36,10
49527,77,37,3137,0,2,13,0,2,1,1,20,3
49528,55,24,0,0,0,11,2,3,0,1,0,3
49529,40,24,0,0,0,10,2,3,0,0,0,9


In [20]:
y = census_data["income"]

## **Training a neural network**

### **Definition of the network**

In [35]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(len(x.columns))),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="sigmoid"),
    keras.layers.Dense(200, activation="sigmoid"),
    keras.layers.Dense(500, activation="sigmoid"),
    keras.layers.Dense(200, activation="sigmoid"),
    keras.layers.Dense(100, activation="sigmoid"),
    keras.layers.Dense(1)
])

In [36]:
opt = keras.optimizers.Adam(learning_rate=0.05)
model.compile(optimizer=opt, loss="mse")
model.fit(x, y, validation_split=0.2, epochs=1000, batch_size=1000)

Train on 39624 samples, validate on 9907 samples
Epoch 1/1000
39624/39624 [==============================] - 1s 26us/sample - loss: 1830163789.1670 - val_loss: 1931148675.4238
Epoch 2/1000
39624/39624 [==============================] - 0s 10us/sample - loss: 1823452514.5391 - val_loss: 1924280750.0991
Epoch 3/1000
39624/39624 [==============================] - 0s 10us/sample - loss: 1816814364.6081 - val_loss: 1917449339.7105
Epoch 4/1000
39624/39624 [==============================] - 0s 10us/sample - loss: 1810214267.4000 - val_loss: 1910676137.9388
Epoch 5/1000
39624/39624 [==============================] - 0s 11us/sample - loss: 1803658145.5183 - val_loss: 1903918752.0161
Epoch 6/1000
39624/39624 [==============================] - 0s 11us/sample - loss: 1797131720.3343 - val_loss: 1897204696.3351
Epoch 7/1000
39624/39624 [==============================] - 0s 11us/sample - loss: 1790634985.3358 - val_loss: 1890537521.0578
Epoch 8/1000
39624/39624 [==============================] - 0s